In [2]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob
import os
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import ElasticVectorSearch
from elasticsearch import Elasticsearch
import numpy as np

In [3]:
# Initialize an empty list to hold all chunks
all_docs = []

# Define the directory where your chapter files are located
chapters_dir = '../data/FCRA Course/modules/'

# Get a list of all chapter files
chapter_files = glob.glob(os.path.join(chapters_dir, '*.txt'))

for file_path in chapter_files:
    # Extract chapter name from file path
    chapter_name = os.path.basename(file_path).replace('.txt', '')

    # Load each chapter
    loader = TextLoader(file_path, encoding='utf-8')
    documents = loader.load()

    # Split the chapter into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    docs = text_splitter.split_documents(documents)

    # Add chapter metadata to each chunk
    for doc in docs:
        doc.metadata['chapter'] = chapter_name

    # Add the chunks to the all_docs list
    all_docs.extend(docs)

# Output the total number of chunks created
print(f"Total number of chunks created: {len(all_docs)}")

Total number of chunks created: 718


In [4]:
#  Load environment variables
load_dotenv()

# Initialize the embedding model with the API key
embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Generate embeddings for each chunk (Optional Step)
texts = [doc.page_content for doc in all_docs]
metadatas = [doc.metadata for doc in all_docs]

# Connect to Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

# Create the vector store in Elasticsearch
vector_store = ElasticVectorSearch(
    embedding=embedding_model,
    elasticsearch_url="http://localhost:9200",
    index_name="fcra_chunks"
)

# Index the documents
vector_store.add_texts(texts=texts, metadatas=metadatas)

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_98216/1540478151.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_98216/1540478151.py:15: LangChainPendingDeprecationWarning: The class `ElasticVectorSearch` will be deprecated in a future version. Use :class:`~Use ElasticsearchStore class in langchain-elasticsearch package` instead.
  vector_store = ElasticVectorSearch(
/Users/hani/Documents/git/llm-zoomcamp-project/llm_project/lib/python3.11/site-packages/langchain_community/vectorstores/elastic_vector_search.py:148: UserWarning:

['f28d9c1e-de35-4f1a-b652-43e9da24435c',
 '9b025eae-4cfa-4b2c-a4fe-b19290f718fe',
 '3b1af43f-cf05-4bd4-8701-04b985c70bb0',
 'a3c37bdc-f353-418e-9afc-de732d4eec02',
 '35029bd4-32d5-4c16-9013-3a990696a7ed',
 '99b86c05-74e9-45bf-8a1e-c06aaa4fa96b',
 '4868df7c-2cd2-4ab4-a507-e8d4e1303e8e',
 '2dd168cc-7e11-4dab-922f-d400690009ae',
 'f373f717-e592-4d72-a562-41074b6ed802',
 'c8d1e3c2-aedb-4ad4-90a6-39c78e7c0ffc',
 'f648ad32-6d5e-4cc4-8986-20afbcacac01',
 '49869e9f-21c3-41e7-b819-ff80ed1e7ea2',
 '99420315-c70b-48fd-88db-d2936c516fb7',
 'b8a4d747-698e-4767-81c1-15de4dc36e96',
 '62046714-e6b8-4f6e-9f0f-0b9c29360165',
 'f09d0373-1851-4f77-b23b-3aa5e10ed34b',
 '926f699f-1ab9-4583-8eb5-f8323f2abfa6',
 '11c05184-bc91-44d2-be08-3eca560b49d6',
 '2529da0f-ee70-4094-b5b5-d4b46ce3f7ff',
 'aade4fc6-7d9f-4e7c-b364-cf53a47fb77e',
 '6ea5e539-c633-43e8-a9eb-105e9703efe7',
 '96a00016-6158-4a2c-af79-45ffe3b039e7',
 '51e8464c-31b6-4dbc-a363-27a2ac48538b',
 '58ddf8da-5c8f-43c4-8ef2-321107b275e0',
 '5c2c0687-af0e-

In [5]:
index_name = 'fcra_chunks'

# Delete the index if it already exists (optional)
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Define the mapping
mapping = {
    "mappings": {
        "properties": {
            "embedding": {
                "type": "dense_vector",
                "dims": 1536
            },
            "text": {
                "type": "text"
            },
            "metadata": {
                "type": "object",
                "enabled": True
            }
        }
    }
}

# Create the index with the mapping
es.indices.create(index=index_name, body=mapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'fcra_chunks'})

## Indexing Document Chunks and Embeddings

In [6]:
# Generate embeddings for each chunk
embeddings = embedding_model.embed_documents(texts)

In [7]:
# Since we’ll be using cosine similarity in script scoring, it’s important to normalize the embeddings.
# Normalize embeddings
embeddings = [embedding / np.linalg.norm(embedding) for embedding in embeddings]

In [8]:
# We’ll prepare a list of actions to bulk index the data into Elasticsearch.

from elasticsearch.helpers import bulk

# Prepare actions for bulk indexing
actions = [
    {
        "_index": index_name,
        "_id": str(i),
        "_source": {
            "embedding": embedding.tolist(),  # Convert numpy array to list
            "text": text,
            "metadata": metadata
        }
    }
    for i, (embedding, text, metadata) in enumerate(zip(embeddings, texts, metadatas))
]

# Bulk index the documents
bulk(es, actions)

(718, [])

In [27]:
# Save Embeddings and Metadata

import json

# Assuming 'texts', 'embeddings', and 'metadatas' are the lists of document content, embeddings, and metadata
data_to_save = []
for text, embedding, metadata in zip(texts, embeddings, metadatas):
    data_to_save.append({
        'text': text,
        'embedding': embedding.tolist(),  # Convert numpy array to list
        'metadata': metadata
    })

# Save to a JSON file
with open('document_embeddings.json', 'w') as f:
    json.dump(data_to_save, f)

 ## Implementing Similarity Search with Script Scoring

In [9]:
# Since we cannot use the knn query without the appropriate plugin, we’ll use script scoring to perform similarity search.

# Define a Function to Normalize Vectors
def normalize_vector(vector):
    norm = np.linalg.norm(vector)
    if norm == 0:
        return vector
    return vector / norm

In [10]:
# Prepare the Query Embedding
    
# User query
# user_query = "What are the key provisions of the FCRA?"
user_query = "What are the permissible purposes for obtaining a consumer report?"

# Generate and normalize the query embedding
query_embedding = embedding_model.embed_query(user_query)
query_embedding = normalize_vector(query_embedding)

In [11]:
# Build the script score query
script_query = {
    "size": 5,
    "query": {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {
                    "query_vector": query_embedding.tolist()
                }
            }
        }
    },
    "_source": ["text", "metadata"]
}

In [12]:
# Execute the Search

response = es.search(index=index_name, body=script_query)

In [13]:
# Display the results
for hit in response['hits']['hits']:
    print("Score:", hit['_score'])
    print("Text snippet:", hit['_source']['text'][:200])  # Display first 200 characters
    print("Metadata:", hit['_source']['metadata'])
    print("------")

Score: 1.901682
Text snippet: authorized under the FCRA. These other purposes include obtaining certain

government benefits, renting an apartment, opening a bank account, or

cashing a check. Thus, if information is expected to b
Metadata: {'chapter': 'Module 02', 'source': '../data/FCRA Course/modules/Module 02.txt'}
------
Score: 1.8994259
Text snippet: under this section to obtain a consumer report on the borrower in that transaction.

Section 604(a)(3)(F) - The catch-all phrase which concludes the list of permissible

purposes is "legitimate busine
Metadata: {'chapter': 'Module 03', 'source': '../data/FCRA Course/modules/Module 03.txt'}
------
Score: 1.8970131
Text snippet: of consumer report information. That is, you may obtain a consumer report to

underwrite a consumer's application for credit or insurance, but you generally

may not obtain a consumer report for law e
Metadata: {'chapter': 'Module 02', 'source': '../data/FCRA Course/modules/Module 02.txt'}
------
Score: 1.89438

## Integrating Retrieval with the GPT API

In [14]:
from langchain.llms import OpenAI

# Initialize the LLM
llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_98216/1808343104.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(openai_api_key=os.getenv("OPENAI_API_KEY"))


In [20]:
# Create a Custom Retriever Using Elasticsearch

from langchain.schema import Document
from langchain.schema import BaseRetriever
from typing import Any, List
import numpy as np
from pydantic import BaseModel

class ElasticSearchRetriever(BaseRetriever, BaseModel):
    es: Any
    index_name: str
    embedding_model: Any
    k: int = 5

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Generate and normalize the query embedding
        query_embedding = self.embedding_model.embed_query(query)
        query_embedding = query_embedding / np.linalg.norm(query_embedding)

        # Build the script score query
        script_query = {
            "size": self.k,
            "query": {
                "script_score": {
                    "query": {"match_all": {}},
                    "script": {
                        "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                        "params": {
                            "query_vector": query_embedding.tolist()
                        }
                    }
                }
            },
            "_source": ["text", "metadata"]
        }

        # Execute the search
        response = self.es.search(index=self.index_name, body=script_query)

        # Convert hits to Documents
        docs = []
        for hit in response['hits']['hits']:
            doc = Document(
                page_content=hit['_source']['text'],
                metadata=hit['_source']['metadata']
            )
            docs.append(doc)
        return docs

# Initialize the Retriever
retriever = ElasticSearchRetriever(
    es=es,
    index_name=index_name,
    embedding_model=embedding_model,
    k=5  # Number of documents to retrieve
)

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_98216/3366245207.py:7: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class ElasticSearchRetriever(BaseRetriever, BaseModel):


In [23]:
# Create the RetrievalQA chain using the from_chain_type method
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff",  # You can use "stuff", "map_reduce", etc., depending on your use case
    retriever=retriever
)

In [24]:
# Run a test query
query = "What are the key provisions of the FCRA?"

# Get the answer from the chain
answer = qa_chain.run(query)

# Print the answer
print("Answer:", answer)

/var/folders/bs/rk6rcnf923l4k8xms3k927hr0000gn/T/ipykernel_98216/576171932.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


Answer:  The key provisions of the FCRA include providing consumers with rights to access and correct their data held by consumer reporting agencies, imposing obligations on those taking "adverse action," and defining the class of data that is regulated under the law. It also outlines duties for consumer reporting agencies to protect consumer data, adhere to permissible purpose requirements, and maintain reasonable data accuracy. Additionally, the law sets regulations for the disposal of records by various federal agencies.


In [26]:
queries = [
    "What are the permissible purposes for obtaining a consumer report?",
    "Explain the dispute process under the FCRA.",
    "What obligations do credit reporting agencies have according to the FCRA?"
]

for query in queries:
    answer = qa_chain.run(query)
    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print("------")

Query: What are the permissible purposes for obtaining a consumer report?
Answer:  The permissible purposes for obtaining a consumer report include obtaining government benefits, renting an apartment, opening a bank account, cashing a check, and underwriting a consumer's application for credit or insurance. Other permissible purposes include a consumer's initiation of a transaction or the use of a consumer's account by the entity holding it. The catch-all phrase "legitimate business need for information" is also a permissible purpose, but only if it involves eligibility determinations for the consumer.
------
Query: Explain the dispute process under the FCRA.
Answer:  The dispute process under the FCRA allows consumers to request a free copy of their credit report from a consumer reporting agency and dispute any inaccurate or incomplete information. The consumer has the right to dispute the information with the consumer reporting agency and the furnisher of the information, such as a l